In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

--2023-05-20 22:57:46--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.67.246.47, 18.67.246.176, 18.67.246.167, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.67.246.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  5.37MB/s    in 7.3s    

2023-05-20 22:57:53 (4.98 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]



In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-20 22:58:14--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.67.246.47, 18.67.246.176, 18.67.246.167, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.67.246.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45616512 (44M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-02.parquet’

yellow_tripdata_202 100%[===================>]  43.50M  5.71MB/s    in 9.7s    

2023-05-20 22:58:24 (4.49 MB/s) - ‘yellow_tripdata_2022-02.parquet’ saved [45616512/45616512]



In [107]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [122]:
import warnings
warnings.filterwarnings("ignore")

In [49]:
df = pd.read_parquet("yellow_tripdata_2022-01.parquet")

In [50]:
# Question 1
df.shape

(2463931, 19)

In [51]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [52]:
df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])

In [53]:
df["duration"] = df["duration"].apply(lambda x:x.total_seconds() / 60)

In [54]:
df[["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "duration"]].head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,37.533333


In [55]:
# Question 2
df["duration"].describe(percentiles=[0.01, 0.99]).round(2)

count    2463931.00
mean          14.21
std           46.45
min        -3442.40
1%             0.58
50%           10.18
99%           50.85
max         8513.18
Name: duration, dtype: float64

In [56]:
all_records = df.shape[0]

In [87]:
df = df.query("duration>=1 and duration<=60")

In [88]:
# Question 3 
df.shape[0]/all_records * 100

98.27547930522405

In [96]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [119]:
categorical = ["PULocationID", "DOLocationID"]

In [97]:
for x in categorical:
    df[x] = df[x].astype(str)

In [98]:
dv = DictVectorizer()

In [99]:
train_dict = df[categorical].to_dict(orient="records")

In [100]:
dv = DictVectorizer()

In [110]:
X_train = dv.fit_transform(train_dict)
y_train = df.duration.values

In [111]:
# Question 4 
len(dv.feature_names_)

515

In [117]:
# Question 5 
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190841034851

In [120]:
categorical = ["PULocationID", "DOLocationID"]

def transform_df(data):
    df = pd.read_parquet(data)
    
    df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
    df["duration"] = df["duration"].apply(lambda x:x.total_seconds() / 60)
    df = df.query("duration>=1 and duration<=60")
    
    for x in categorical:
        df[x] = df[x].astype(str)
        
    return df 

In [123]:
df_2 = transform_df("yellow_tripdata_2022-02.parquet")

In [125]:
val_dict = df_2[categorical].to_dict(orient="records")

In [126]:
X_val = dv.transform(val_dict)
y_val = df_2.duration.values

In [127]:
# Question 6 
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786408695263233